In [1]:
import os

In [2]:
%pwd

'd:\\Data_science\\Projects\\WineQulity_ml\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Data_science\\Projects\\WineQulity_ml'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    """
    Configuration for the data ingestion stage.
    """
    root_dir: Path  # Root directory where data ingestion artifacts are stored
    source_URL: str  # URL to the source data file
    local_data_file: Path  # Local path to save the downloaded data file
    unzip_dir: Path  # Directory where the data will be unzipped

In [6]:
from WineMl.constants import *
from WineMl.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_URL= config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [8]:
import os
from pathlib import Path
import zipfile
from WineMl import logger
from WineMl.utils.common import get_size
import requests
from tqdm import tqdm

In [9]:
class DataIngestion:
    def __init__(self, config : DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        # The file will only be download if it dosn't already exist
        if not os.path.exists(self.config.local_data_file):
            logger.info("Starting file download...")

            # ---- START: Progress Bar IMplementation ----
            url = self.config.source_URL
            local_filename = self.config.local_data_file


            # 1. Initiate the request with streming enabled
            response = requests.get(url, stream=True)
            response.raise_for_status() #Checl for bad status

            # 2. Get the total file size(Content-lenth)
            total_size_in_bytes = int(response.headers.get('content-length', 0))
            block_size = 1024 # 1 kilobyte


            # 3. use tqdm to track progress
            progress_bar = tqdm(
                total=total_size_in_bytes,
                unit='iB',
                unit_scale=True,
                desc = local_filename
            )

            # 4. write to  file and update the progress bar
            with open(local_filename, 'wb') as file:
                for data in response.iter_content(block_size):
                    progress_bar.update(len(data))
                    file.write(data)

            progress_bar.close()
            # ------ END: Progress bar Implementation ---

            # Check if download was complete (optional integrity check)
            if total_size_in_bytes !=0 and progress_bar.n != total_size_in_bytes:
                logger.warning("Download integrity check failed. File Size mismatch.")


            logger.info(f"{local_filename} downloaded successfully!")

        else: 
            # Original logic for file already existing
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
    

    def extract_zip_file(self):

        """
        Extracts a zip file into a specified directory.
        
        This method takes the path of a downloaded zip file and extracts its
        contents to the designated `unzip_dir`. It ensures the destination
        directory exists before extraction.
        """
        unzip_path = self.config.unzip_dir

        os.makedirs(unzip_path, exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        
        logger.info(f"Zip file extracted to {unzip_path}")


In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2025-09-29 02:19:52,697: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-29 02:19:52,699: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-29 02:19:52,701: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-09-29 02:19:52,703: INFO: common: created directory at: artifacts_root]
[2025-09-29 02:19:52,705: INFO: common: created directory at: artifacts/data_ingestion]
[2025-09-29 02:19:52,706: INFO: 3978221524: Starting file download...]


artifacts/data_ingestion/data.zip: 100%|██████████| 23.3k/23.3k [00:00<00:00, 708kiB/s]

[2025-09-29 02:19:53,753: INFO: 3978221524: artifacts/data_ingestion/data.zip downloaded successfully!]
[2025-09-29 02:19:53,782: INFO: 3978221524: Zip file extracted to artifacts/data_ingestion]
